In [216]:
!pip install mpld3
import mpld3
mpld3.enable_notebook()
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
            
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)
            
#import the necessary modelling algos.
            
#classifiaction.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
            
#regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
            
#model selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
            
#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification

In [0]:
dlog = pd.read_csv("/home/log_feature.csv")
dtrain = pd.read_csv("/home/train.csv")
dseverity = pd.read_csv("/home/severity_type.csv")
dtest = pd.read_csv("/home/test.csv")
devent = pd.read_csv("/home/event_type.csv")
dresource = pd.read_csv("/home/resource_type.csv")

In [218]:
dlog.head(5)

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1
3,5022,feature 172,2
4,5022,feature 56,1


In [219]:
dtrain.head(5)

,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0


In [0]:
# dresource.head(5)

In [0]:
# dresource["id"], dresource["resource_type"]= dresource["id,resource_type"].str.split(",",1).str
# dresource.drop("id,resource_type",  axis=1, inplace=True)
# dresource.head(5)

In [0]:
# devent.head(5)

In [0]:
# devent["id"], devent["event_type"]= devent[0].str.split(",",1).str
# devent.drop([0],  axis=1, inplace=True)
# devent.drop(devent.index[[0]], inplace=True)  #first row

In [224]:

devent.head(5)

,id,event_type
0,6597,event_type 11
1,8011,event_type 15
2,2597,event_type 15
3,5022,event_type 15
4,5022,event_type 11


In [225]:
dseverity.head(5)

,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1


In [226]:
columns = [dtrain.columns, dlog.columns, devent.columns, dseverity.columns,dresource.columns, dtest.columns]
shape = [dtrain.shape, dlog.shape, devent.shape, dseverity.shape,dresource.shape, dtest.shape]
columns


[Index(['id', 'location', 'fault_severity'], dtype='object'),
 Index(['id', 'log_feature', 'volume'], dtype='object'),
 Index(['id', 'event_type'], dtype='object'),
 Index(['id', 'severity_type'], dtype='object'),
 Index(['id', 'resource_type'], dtype='object'),
 Index(['id', 'location'], dtype='object')]

In [227]:
shape

[(7381, 3), (58671, 3), (31170, 2), (18552, 2), (21076, 2), (11171, 2)]

In [0]:
# plt.scatter( dseverity.id,dseverity.severity_type)

In [0]:
# def concat_data(a,b):
#   res = []
#   for id in a:
#     if(a.id & b.id & a.id == b.id):
#       res.append(a.iloc[1:2], b.iloc[1:2])
#   return res;
# c = concat_data(dseverity, devent);
# c.head(5)

In [230]:
devent.head(2)

,id,event_type
0,6597,event_type 11
1,8011,event_type 15


In [231]:

devent.event_type.value_counts().max()

7888

In [0]:
# msno.matrix(devent)

In [233]:
devent.isnull().sum()

id            0
event_type    0
dtype: int64

In [234]:
dtrain["source"] = "train"
dtest["source"] = "test"
data = pd.concat([dtrain, dtest], ignore_index=True)
data.head(5)

,fault_severity,id,location,source
0,1.0,14121,location 118,train
1,0.0,9320,location 91,train
2,1.0,14394,location 152,train
3,1.0,8218,location 931,train
4,0.0,14804,location 120,train


In [235]:
devent = devent.merge(data, on='id')
devent.head()

,id,event_type,fault_severity,location,source
0,6597,event_type 11,NaN,location 1,test
1,8011,event_type 15,0.0,location 1,train
2,2597,event_type 15,NaN,location 1,test
3,5022,event_type 15,NaN,location 1,test
4,5022,event_type 11,NaN,location 1,test


In [236]:
eventq = pd.DataFrame(devent.event_type.value_counts())
eventq.head()

,event_type
event_type 11,7888
event_type 35,6615
event_type 34,5927
event_type 15,4395
event_type 20,1458


In [237]:
## of train set
eventq["percTain"] = devent.pivot_table(values="source", index="event_type",aggfunc=lambda x: sum(x == "train")/float(len(x)))
eventq.head()

,event_type,percTain
event_type 11,7888,0.388945
event_type 35,6615,0.407105
event_type 34,5927,0.406783
event_type 15,4395,0.392264
event_type 20,1458,0.382030


In [238]:
devent.head()

,id,event_type,fault_severity,location,source
0,6597,event_type 11,NaN,location 1,test
1,8011,event_type 15,0.0,location 1,train
2,2597,event_type 15,NaN,location 1,test
3,5022,event_type 15,NaN,location 1,test
4,5022,event_type 11,NaN,location 1,test


In [0]:
from scipy.stats.mstats import mode


In [240]:
# mode
eventq["mode_severity"] = devent.loc[devent["source"]=="train"].pivot_table(index="event_type", values="fault_severity", aggfunc=lambda x: mode(x).mode[0])
eventq.tail()

,event_type,percTain,mode_severity
event_type 48,2,0.5,0.0
event_type 41,2,0.5,0.0
event_type 4,1,0.0,NaN
event_type 33,1,0.0,NaN
event_type 52,1,0.0,NaN


In [241]:
eventq["preprocess"] = eventq.index.values
eventq.head()

,event_type,percTain,mode_severity,preprocess
event_type 11,7888,0.388945,0.0,event_type 11
event_type 35,6615,0.407105,0.0,event_type 35
event_type 34,5927,0.406783,0.0,event_type 34
event_type 15,4395,0.392264,0.0,event_type 15
event_type 20,1458,0.382030,0.0,event_type 20


In [242]:
#remove the ones not present in train:
eventq["preprocess"].loc[eventq["mode_severity"] == 0] = "remove"
eventq.head()

,event_type,percTain,mode_severity,preprocess
event_type 11,7888,0.388945,0.0,remove
event_type 35,6615,0.407105,0.0,remove
event_type 34,5927,0.406783,0.0,remove
event_type 15,4395,0.392264,0.0,remove
event_type 20,1458,0.382030,0.0,remove


In [0]:
#replace the lower ones with mode:
top_unchange = 3
eventq['preprocess'].iloc[top_unchange:] = eventq['mode_severity'].iloc[top_unchange:].apply(lambda x: 'Remove' if pd.isnull(x) else 'event_type others_%d'%int(x))

In [244]:
devent = devent.merge(eventq[['preprocess']], left_on='event_type',right_index=True)
devent.head()

,id,event_type,fault_severity,location,source,preprocess
0,6597,event_type 11,NaN,location 1,test,remove
4,5022,event_type 11,NaN,location 1,test,remove
5,6852,event_type 11,NaN,location 1,test,remove
9,14838,event_type 11,NaN,location 1,test,remove
11,2588,event_type 11,0.0,location 1,train,remove


In [245]:
devent_merge = devent.pivot_table(index="id", values="event_type",columns="preprocess", aggfunc=lambda x: len(x),fill_value=0)
devent_merge.head()

preprocess,Remove,event_type others_0,event_type others_1,event_type others_2,remove
id,,,,,
1,0,0,1,0,1
2,0,0,0,0,2
3,0,0,0,0,1
4,0,1,0,0,0
5,0,0,0,0,2


In [0]:
data  = data.merge(devent_merge, left_on="id", right_index=True)

In [205]:
data.head()

,fault_severity,id,location,source,Remove,event_type 10,event_type 13,event_type 2,event_type 22,event_type 28,event_type 29,event_type 30,event_type 5,event_type 6,event_type 7,event_type others_0,event_type others_1,event_type others_2,remove
0,1.0,14121,location 118,train,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0.0,9320,location 91,train,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,1.0,14394,location 152,train,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,1.0,8218,location 931,train,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,0.0,14804,location 120,train,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3


In [207]:
devent_merge.shape

(18552, 15)